In [ ]:
In this notebook, we use [Simon Tatham's work](http://www.chiark.greenend.org.uk/~sgtatham/polyhedra/) on polyhedra as a base to create a blastula-like epithelium (make sure to explore Simon's website, I'll wait here).

### Imports

In [10]:
import numpy as np, pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from IPython.display import display

from tyssue import config, Epithelium, Sheet
from tyssue import SheetGeometry as sgeom, BulkGeometry as geom
from tyssue.core.generation import extrude

from tyssue.draw.plt_draw import sheet_view
from tyssue.draw.threejs_draw import view_3js


#### File parser for S. Thatam polyhedra files 

In [2]:
# What to we have?
!ls ../data/polyhedra/

cube			     octahedron			  triakisicosahedron
cuboctahedron		     pentakisdodecahedron	  triakisoctahedron
dodecahedron		     rhombicdodecahedron	  triakistetrahedron
greatrhombicosidodecahedron  rhombictriacontahedron	  truncatedcube
greatrhombicuboctahedron     smallrhombicosidodecahedron  truncateddodecahedron
hexakisicosahedron	     smallrhombicuboctahedron	  truncatedicosahedron
hexakisoctahedron	     tetrahedron		  truncatedoctahedron
icosahedron		     tetrakishexahedron		  truncatedtetrahedron
icosidodecahedron	     trapezoidalhexecontahedron
LICENCE			     trapezoidalicositetrahedron


The file structure is prety strait forward:

In [3]:
%%bash
head ../data/polyhedra/truncatedicosahedron

point A -0.53934466291663152 -0.66666666666666674 -1.4120226591665967
point B -1.0786893258332635 -0.33333333333333326 -1.2060113295832982
point C 0.0 -0.33333333333333348 -1.6180339887498949
point D 0.0 0.33333333333333348 -1.6180339887498949
point E 0.53934466291663152 0.66666666666666674 -1.4120226591665967
point F 1.0786893258332635 0.33333333333333326 -1.2060113295832982
point G 0.0 -0.33333333333333348 1.6180339887498949
point H 0.0 0.33333333333333348 1.6180339887498949
point I -0.53934466291663152 0.66666666666666674 1.4120226591665967
point J -1.0786893258332635 0.33333333333333326 1.2060113295832982


In [4]:
%%bash
tail ../data/polyhedra/truncatedicosahedron

face JBfAdZAr Ad
face JBfAdZAr Z
face JBfAdZAr Ar
normal JBfAdZAr -0.85065080835203999 0.0 0.52573111211913359
face LAvAhAjBh L
face LAvAhAjBh Av
face LAvAhAjBh Ah
face LAvAhAjBh Aj
face LAvAhAjBh Bh
normal LAvAhAjBh 0.85065080835203999 0.0 0.52573111211913359


In [51]:
def parse_polyhedron(fname):
    '''
    Parses a polynome description provided by Simon Tatham
    
    Polyhedron files are retrieved from
    http://www.chiark.greenend.org.uk/~sgtatham/polyhedra/
    
    Parameters
    ----------
    fname: str or path
      The polyhedron file to parse
    
    Returns
    -------
    datasets: dict
      The vert, edge and face datasets


    The software in this archive is copyright 2003,2004 Simon Tatham.

    Permission is hereby granted, free of charge, to any person
    obtaining a copy of this software and associated documentation files
    (the "Software"), to deal in the Software without restriction,
    including without limitation the rights to use, copy, modify, merge,
    publish, distribute, sublicense, and/or sell copies of the Software,
    and to permit persons to whom the Software is furnished to do so,
    subject to the following conditions:

    The above copyright notice and this permission notice shall be
    included in all copies or substantial portions of the Software.

    THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
    EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
    MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
    NONINFRINGEMENT.  IN NO EVENT SHALL THE COPYRIGHT HOLDERS BE LIABLE
    FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF
    CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION
    WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
    '''
    
    
    with open(fname, 'r+') as th:
        lines = th.readlines()

    point_lines = [l.split(' ') for l in lines if l.startswith('point')]
    # point Bh 1.0786893258332635 0.33333333333333326 1.2060113295832982

    points = np.array([[np.float(s) for s in p[-3:]] for p in point_lines])
    point_ids = [p[1] for p in point_lines]

    vert_df = pd.DataFrame(points, index=pd.Index(point_ids), columns=['x', 'y', 'z'])
    vert_df['int_idx'] = np.arange(vert_df.shape[0])

    face_lines = [l.split(' ')[1:] for l in lines if l.startswith('face')]
    # ABAxAwDC A
    prev = face_lines[0][0] # ABAxAwDC
    faces = []
    cur = []
    for ff, l in face_lines:
        if ff != prev:
            faces.append(cur)
            cur = [l[:-1]]
            prev = ff
        else:
            cur.append(l[:-1])
    
    faces.append(cur)
    
    print(len(faces))

    srces = []
    trgts = []
    face_idxs = []
    for i, ff in enumerate(faces):
        srces.extend(vert_df.loc[ff, 'int_idx'])
        trgts.extend(vert_df.loc[np.roll(ff, 1), 'int_idx'])
        face_idxs.extend(np.ones(len(ff))*i)

    edge_df = pd.DataFrame(np.vstack([srces, trgts, face_idxs]).T.astype(np.int),
                           columns=['srce', 'trgt', 'face'])
    edge_df.index.name = 'edge'
    vert_df['letter'] = vert_df.index
    vert_df.set_index('int_idx', drop=True, inplace=True)
    vert_df.index.name = 'vert'
    face_df = pd.DataFrame(index=pd.Index(range(len(faces)), name='face'),
                           columns=['x', 'y', 'z'])

    datasets = {'vert': vert_df,
                'edge': edge_df,
                'face': face_df}
    return datasets


In [52]:
apical_datasets['vert']

,x,y,z,letter,rho,basal_shift,height,is_active,radial_tension
vert,,,,,,,,,
0,-0.539345,-0.666667,-1.412023,A,1.652012,4.0,-2.347988,True,0.0
1,-1.078689,-0.333333,-1.206011,B,1.652012,4.0,-2.347988,True,0.0
2,0.000000,-0.333333,-1.618034,C,1.652012,4.0,-2.347988,True,0.0
3,0.000000,0.333333,-1.618034,D,1.652012,4.0,-2.347988,True,0.0
4,0.539345,0.666667,-1.412023,E,1.652012,4.0,-2.347988,True,0.0
5,1.078689,0.333333,-1.206011,F,1.652012,4.0,-2.347988,True,0.0
6,0.000000,-0.333333,1.618034,G,1.652012,4.0,-2.347988,True,0.0
7,0.000000,0.333333,1.618034,H,1.652012,4.0,-2.347988,True,0.0
8,-0.539345,0.666667,1.412023,I,1.652012,4.0,-2.347988,True,0.0


## Truncated Icosahedron (or fullerene)

In [53]:
apical_datasets = parse_polyhedron('../data/polyhedra/truncatedicosahedron')
specs = config.geometry.spherical_sheet()

sheet = Sheet('sphere', apical_datasets, specs)
mod_specs = config.dynamics.quasistatic_sheet_spec()
sheet.update_specs(mod_specs)
sgeom.update_all(sheet)
#res = solver.find_energy_min(sheet, sgeom, model)
#print(res['success'])

32


In [54]:
render, lines = view_3js(sheet)
display(render)

In [55]:
from tyssue.io import hdf5
hdf5.save_datasets('../data/hf5/fullerene.hf5', sheet)

### Creating a monolayer through extrusion

In [16]:
apical_datasets = parse_polyhedron('../data/polyhedra/truncatedicosahedron')
datasets = extrude(apical_datasets, scale=0.8)
specs = config.geometry.bulk_spec()

eptm = Epithelium('truncatedicosahedron', datasets, specs)
geom.update_all(eptm)

In [17]:
render2, lines = view_3js(eptm)
display(render2)

#### Try out others! 

In [18]:
apical_datasets = parse_polyhedron(
    '../data/polyhedra/trapezoidalicositetrahedron')
specs = config.geometry.spherical_sheet()

sheet = Sheet('sphere', apical_datasets, specs)
mod_specs = config.dynamics.quasistatic_sheet_spec()
sheet.update_specs(mod_specs)
sgeom.update_all(sheet)

render3, lines = view_3js(sheet)
display(render3)

In [19]:
apical_datasets = parse_polyhedron(
    '../data/polyhedra/greatrhombicosidodecahedron')
specs = config.geometry.spherical_sheet()

sheet = Sheet('sphere', apical_datasets, specs)
mod_specs = config.dynamics.quasistatic_sheet_spec()
sheet.update_specs(mod_specs)
sgeom.update_all(sheet)

render3, lines = view_3js(sheet)
display(render3)